In [ ]:
import os
import sys

from dotenv import load_dotenv
import pandas as pd
import requests

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.etl.extractors.strava import (
    StravaEndpoints,
    StravaExtractor,
    get_fresh_access_token,
)

In [ ]:
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
REFRESH_TOKEN = os.getenv('REFRESH_TOKEN')

In [ ]:
activities_url = StravaEndpoints.get_activities()
access_token = get_fresh_access_token(CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN)

header = {'Authorization': f'Bearer {access_token}'}

In [ ]:
request_page_num = 1
all_activities: list[str] = []

while True:
    param = {'per_page': 200, 'page': request_page_num}
    my_dataset = requests.get(activities_url, headers=header, params=param).json()

    if len(my_dataset) == 0:
        break

    if all_activities:
        all_activities.extend(my_dataset)

    else:
        all_activities = my_dataset

    request_page_num += 1

In [ ]:
extractor = StravaExtractor(access_token)
all_activities_new = extractor.fetch_all_activities()
all_activities_new

In [ ]:
len(all_activities)

In [ ]:
df_all_activities = pd.json_normalize(all_activities)
df_all_activities.head()

# Exploring columns

In [ ]:
df_all_activities.columns

In [ ]:
df_all_activities.dtypes

In [ ]:
with pd.option_context('display.max_columns', None):
    display(df_all_activities.head())

In [ ]:
df_all_activities.has_kudoed.unique()

## Filter out unimportant columns

In [ ]:
unimportant_columns = [
    'resource_state',
    'location_city',
    'location_state',
    'location_country',
    'commute',
    'flagged',
    'start_latlng',
    'end_latlng',
    'heartrate_opt_out',
    'external_id',
    'upload_id',
    'upload_id_str',
    'from_accepted_tag',
    'has_kudoed',
    'athlete_resource_state',
    'map_resource_state',
]

In [ ]:
df_trunc = df_all_activities.drop(columns=unimportant_columns, errors='ignore')
df_trunc